# Get the response matrix from a ROOT ntuple
Define the input file and its metadata

In [1]:
path = "~/tau_pipi0_BGx1.root"
variable_title = "invariant mass"
tree_name = "tau"

Define the fields to load as the truth and the simulated variables

In [7]:
import math

def get_truth(event):
    return event.tau_0_matchedMC_InvM

def get_detected(event):
    return event.tau_0_M

## Histograms generation
Iterate entry by entry to build the histograms

In [8]:
import ROOT
import time

# Source file
chain = ROOT.TChain(tree_name)
chain.AddFile(path)

# Define the histograms metadata
bins = 50
rmin = 0.2
rmax = 1.8

# Define the histograms
hist_mc = ROOT.TH1D("truth", f"Truth {variable_title}", bins, rmin, rmax)
hist_detected = ROOT.TH1D("detected", f"Detected {variable_title}", bins, rmin, rmax)
response = ROOT.TH2D(
    "response", f"Response matrix of {variable_title}", bins, rmin, rmax, bins, rmin, rmax
)

start = time.time()
# Iterate all events
for i in range(chain.GetEntries()):
    # Retrieve event to be accessible by the chain
    chain.GetEntry(i)
    # Add the value as generated by the simulation
    truth = get_truth(chain)
    hist_mc.Fill(truth)
    # Add the value as detected/reconstructed by the experiment
    detected = get_detected(chain)
    hist_detected.Fill(detected)
    # Add the value to the response matrix
    response.Fill(detected, truth)
end = time.time()
print(f"Ellapsed time: {end - start} s")

Ellapsed time: 1.622943639755249 s
Warning in <TROOT::Append>: Replacing existing TH1: truth (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: detected (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: response (Potential memory leak).


## Storage
Store the histograms and the response matrix in a new root file

In [9]:
storage_path = "~/response_tau_pipi0_BGx1.root"

In [10]:
storage = ROOT.TFile(storage_path, "RECREATE")
hist_mc.Write()
hist_detected.Write()
response.Write()
storage.Close()
del storage

## Reading and plotting
Read the histograms from the root file and plot them

In [ ]:
storage = ROOT.TFile(storage_path, "READ")
mc_histogram = storage.truth
det_histogram = storage.detected
response_histogram = storage.response